In [1]:
import sys
import os
notebook_path = os.path.abspath('')
project_root = os.path.dirname(os.path.dirname(notebook_path))
sys.path.append(project_root)
from src.transformation import data_collection,data_preprocessing

import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import (
    Input, Conv1D, MaxPooling1D, Dense,Bidirectional,LSTM,Attention
)
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l1, l2
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import os

In [ ]:
def sign_accuracy(y_true, y_pred):
    import tensorflow as tf
    # Compute change between last and first future prices
    y_true_diff = y_true[:, -1] - y_true[:, 0]
    y_pred_diff = y_pred[:, -1] - y_pred[:, 0]

    # Compute sign of differences
    y_true_sign = tf.sign(y_true_diff)
    y_pred_sign = tf.sign(y_pred_diff)

    # Compare signs
    correct_predictions = tf.equal(y_true_sign, y_pred_sign)
    # Convert booleans to floats and compute mean over batch_size
    accuracy = tf.reduce_mean(tf.cast(correct_predictions, tf.float32))
    return accuracy

def build_model(input_shape):
    """    
    How to Access the Model:

    The model is not directly included in this repository. If you are interested in using or learning more about the stock price prediction model, feel free to reach out to me via email:

    📧 **tywongbn@connect.ust.hk**

    I will be happy to share the model and discuss how it works.
   """
    


    # Build Model
    model = Model(inputs=inputs, outputs=outputs)
    # Compile Model
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse',metrics=[sign_accuracy])
    return model



In [3]:
from tensorflow.keras.callbacks import Callback
class SaveBestModelAfter(Callback):
    def __init__(self, monitor='sign_accuracy', mode='max', save_best_only=True, save_path='best_model.h5', start_epoch=30):
        super(SaveBestModelAfter, self).__init__()
        self.monitor = monitor
        self.mode = mode
        self.save_best_only = save_best_only
        self.save_path = save_path
        self.start_epoch = start_epoch
        self.best = -np.Inf if mode == 'max' else np.Inf  # Initialize best value based on mode
        self.epoch_counter = 0

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        self.epoch_counter += 1

        # Only start saving after the specified epoch (e.g., 30)
        if self.epoch_counter >= self.start_epoch:
            current = logs.get(self.monitor)
            if current is None:
                print(f"Warning: Metric '{self.monitor}' is not available in logs.")
                return

            # Check if the current metric value is better than the previous best
            if (self.mode == 'max' and current > self.best) or (self.mode == 'min' and current < self.best):
                print(f"Epoch {epoch + 1}: {self.monitor} improved from {self.best} to {current}. Saving model.")
                self.best = current
                self.model.save(self.save_path)
            else:
                print(f"Epoch {epoch + 1}: {self.monitor} did not improve from {self.best}.")

In [8]:
HANG_SENG_INDEX="^HSI"
training_data = data_collection.get_stock_data(HANG_SENG_INDEX)
traning_data, price_data = data_preprocessing.create_dataset(training_data, window_size = 25, future_window=5,data_cutting=5)

print(traning_data.shape)
print(price_data.shape)
train_data, val_data, train_labels, val_labels = train_test_split(traning_data, price_data, test_size=0.1,shuffle=False)

model = build_model((train_data.shape[1], train_data.shape[2]))

# Create the custom callback
save_best_after_30 = SaveBestModelAfter(
    monitor='val_sign_accuracy',  # Metric to monitor
    mode='max',               # Save when the metric is maximized
    save_best_only=True,      # Save only if the metric improves
    save_path=f'../../models/{HANG_SENG_INDEX}.h5', # Filepath to save the model
    start_epoch=30            # Start saving after 30 epochs
)

# Train the model with the custom callback
history = model.fit(
    train_data, train_labels,                   # Replace with your training data
    validation_data=(val_data, val_labels),     # Replace with your validation data
    epochs=100,                          # Specify the number of epochs
    batch_size=32,                      # Specify the batch size
    callbacks=[save_best_after_30]      # Pass the custom callback
)

[*********************100%***********************]  1 of 1 completed


(4926, 5)
(970, 25, 17)
(970, 5)
Epoch 1/100
28/28 [==============================] - 6s 57ms/step - loss: 4.0916 - sign_accuracy: 0.4942 - val_loss: 3.3165 - val_sign_accuracy: 0.3281
Epoch 2/100
28/28 [==============================] - 1s 21ms/step - loss: 2.7106 - sign_accuracy: 0.4931 - val_loss: 2.0840 - val_sign_accuracy: 0.3906
Epoch 3/100
28/28 [==============================] - 1s 21ms/step - loss: 1.6390 - sign_accuracy: 0.4940 - val_loss: 1.2379 - val_sign_accuracy: 0.3906
Epoch 4/100
28/28 [==============================] - 1s 21ms/step - loss: 0.9235 - sign_accuracy: 0.5181 - val_loss: 0.6736 - val_sign_accuracy: 0.3281
Epoch 5/100
28/28 [==============================] - 1s 22ms/step - loss: 0.5389 - sign_accuracy: 0.4907 - val_loss: 0.4431 - val_sign_accuracy: 0.7031
Epoch 6/100
28/28 [==============================] - 1s 21ms/step - loss: 0.3822 - sign_accuracy: 0.5009 - val_loss: 0.3466 - val_sign_accuracy: 0.6797
Epoch 7/100
28/28 [==============================] - 1s

c:\Python\Python310\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


28/28 [==============================] - 1s 21ms/step - loss: 0.1159 - sign_accuracy: 0.5236 - val_loss: 0.1778 - val_sign_accuracy: 0.3438
Epoch 32/100
28/28 [==============================] - 1s 21ms/step - loss: 0.1151 - sign_accuracy: 0.5594 - val_loss: 0.1791 - val_sign_accuracy: 0.3203
Epoch 33/100
28/28 [==============================] - 1s 22ms/step - loss: 0.1102 - sign_accuracy: 0.5487 - val_loss: 0.1782 - val_sign_accuracy: 0.3672
Epoch 34/100
28/28 [==============================] - 1s 21ms/step - loss: 0.1083 - sign_accuracy: 0.5541 - val_loss: 0.1822 - val_sign_accuracy: 0.3750
Epoch 35/100
28/28 [==============================] - 1s 21ms/step - loss: 0.1006 - sign_accuracy: 0.5589 - val_loss: 0.1871 - val_sign_accuracy: 0.3906
Epoch 36/100
28/28 [==============================] - 1s 22ms/step - loss: 0.1101 - sign_accuracy: 0.5784 - val_loss: 0.1837 - val_sign_accuracy: 0.3828
Epoch 37/100
28/28 [==============================] - 1s 21ms/step - loss: 0.1055 - sign_accura